In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install h2o

     |████████████████████████████████| 164.8MB 88kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.3-py2.py3-none-any.whl size=164854343 sha256=e93f75aaa76e0d5988583c3e71882a7b1f46eeab51864fbb11e768e567351acb
  Stored in directory: /root/.cache/pip/wheels/4f/bb/82/0988682f9abd0b5e95109a4aae6793dd427173b6dc671d9252
Successfully built h2o


In [ ]:
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
pd.options.display.width = 0
h2o.init()
h2o.connect()

ModuleNotFoundError: ignored

In [ ]:
def split_ingest(df, ratio, na_prob, method):

    def impute(df, method):
        methods = {"reduce": df.dropna(),
                "ffill": df.ffill(axis=0),
                "mean": df.fillna(df.mean()),
                "mode": df.fillna(df.mode().iloc[0])}

        if method == "mean":
            mean = methods[method]
            return impute(mean, "reduce")
        else:
            return methods[method]

    df_origin = pd.read_csv(str("/content/drive/MyDrive/") + df['csv'], low_memory=False)
    print(f"loaded DataFrame: \n{df_origin.head()}")

    q_df = df_origin.drop(columns=df["label"])
    holes_df = q_df.mask(np.random.random(q_df.shape) < na_prob)
    holes_df[df["label"]] = df_origin[df["label"]]

    reduced_df = holes_df.sample(frac=ratio, random_state=0)

    imputed_df = impute(reduced_df, method)

    train = imputed_df.sample(frac=0.8)
    test = imputed_df.drop(train.index)

    trainh2o = h2o.H2OFrame(train)
    testh2o = h2o.H2OFrame(test)

    x = trainh2o.columns
    print(x)
    y = df['label']
    x = x.remove(y)

    if df['type'] == "reg":
        trainh2o[y] = trainh2o[y].asnumeric()
        testh2o[y] = testh2o[y].asnumeric()
    elif df['type'] == "cla":
        trainh2o[y] = trainh2o[y].ascharacter()
        testh2o[y] = testh2o[y].ascharacter()
        trainh2o[y] = trainh2o[y].asfactor()
        testh2o[y] = testh2o[y].asfactor()

    return trainh2o, testh2o, x, y

def read_task_csv():
    df = pd.DataFrame({"csv": ["vgsales_alias.csv"],
                       "label": ["Global_Sales"],
                       "type": ["reg"],
                       "leader_model": ["vgsales_leader"]})

    return df

def do_them_all(ratio, missing_rate, method):
    task_df = read_task_csv()

    for idx, row in read_task_csv().iterrows():
        train, test, x, y = split_ingest(row, ratio, missing_rate, method)
        aml = H2OAutoML(max_models=20, seed=1, max_runtime_secs=86400)
        aml.train(x=x, y=y, training_frame=train)

        h2o.save_model(aml.leader, path=str("/content/drive/MyDrive/") + str(row['leader_model']) + str(missing_rate) + str(method))


# --------------------------------------------

In [ ]:
import time
missing_rates = [0.01, 0.05, 0.1, 0.2]
methods = ["reduce", "ffill", "mean", "mode"]

c = 0
start_whole = time.time()
for i in range(1):
    for e in missing_rates:
        for m in methods:
            print(f"On the round {i}, missing rate {e}, and imputation method {m}")
            print(f" {c}/{30*len(missing_rates)*len(methods)} loops")
            time_round = time.time()
            print(f"Time elapsed so far: {time_round - start_whole}")
            do_them_all(0.1, e, m)
            end_time = time.time()
            print(f"Time took for this round: {end_time - time_round}")

On the round 0, missing rate 0.01, and imputation method reduce
 0/480 loops
Time elapsed so far: 0.01040792465209961
loaded DataFrame: 
    Rank                               Name  ...           Publisher Global_Sales
0   2061                               1942  ...              Capcom         1.00
1   9137       �Shin Chan Flipa en colores!  ...           505 Games         0.14
2  14279  .hack: Sekai no Mukou ni + Versus  ...  Namco Bandai Games         0.03
3   8359         .hack//G.U. Vol.1//Rebirth  ...  Namco Bandai Games         0.17
4   7109       .hack//G.U. Vol.2//Reminisce  ...  Namco Bandai Games         0.23

[5 rows x 7 columns]
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Global_Sales']
AutoML progress: |████████████████████████████████████████████████████████| 100%
Time took for this round: 285.8

In [ ]:
def load_model(path, name):
    model = h2o.load_model(str(path) + str(name))
    print(model)

    return model

load_model('/content/drive/MyDrive/vgsales_leader/' , 'XRT_1_AutoML_20210529_152017')

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  XRT_1_AutoML_20210529_152017


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,42.0,42.0,376002.0,20.0,20.0,20.0,273.0,687.0,463.09525




ModelMetricsRegression: drf
** Reported on train data. **

MSE: 1.9260191103849087
RMSE: 1.387810905845933
MAE: 0.18585217017756295
RMSLE: 0.12284273143890628
Mean Residual Deviance: 1.9260191103849087

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 2.5058360080709736
RMSE: 1.582983262094383
MAE: 0.16367007965360403
RMSLE: 0.10247465276911416
Mean Residual Deviance: 2.5058360080709736

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.16362558,0.089994,0.13096131,0.09693985,0.3220629,0.13818651,0.12997735
1,mean_residual_deviance,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
2,mse,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
3,r2,0.8781441,0.18087395,0.9357897,0.967729,0.5565513,0.9864207,0.94422966
4,residual_deviance,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
5,rmse,0.9155989,1.4421666,0.3471687,0.15291272,3.4921966,0.3001867,0.28553006
6,rmsle,0.10156989,0.015366847,0.08676154,0.084921435,0.10519646,0.12036806,0.110601954



Scoring History: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.16362558,0.089994,0.13096131,0.09693985,0.3220629,0.13818651,0.12997735
1,mean_residual_deviance,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
2,mse,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
3,r2,0.8781441,0.18087395,0.9357897,0.967729,0.5565513,0.9864207,0.94422966
4,residual_deviance,2.5021968,5.4188,0.120526105,0.0233823,12.1954365,0.090112045,0.08152741
5,rmse,0.9155989,1.4421666,0.3471687,0.15291272,3.4921966,0.3001867,0.28553006
6,rmsle,0.10156989,0.015366847,0.08676154,0.084921435,0.10519646,0.12036806,0.110601954



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2021-05-29 15:20:52,5.538 sec,0.0,NaN,NaN,NaN
1,,2021-05-29 15:20:52,5.643 sec,5.0,2.778168,0.358816,7.718215
2,,2021-05-29 15:20:52,5.733 sec,10.0,2.152537,0.276698,4.633417
3,,2021-05-29 15:20:52,5.824 sec,15.0,2.013517,0.264721,4.054252
4,,2021-05-29 15:20:53,5.880 sec,20.0,1.652363,0.225319,2.730302
5,,2021-05-29 15:20:53,5.943 sec,25.0,1.618176,0.215768,2.618492
6,,2021-05-29 15:20:53,6.001 sec,30.0,1.548501,0.211878,2.397855
7,,2021-05-29 15:20:53,6.077 sec,35.0,1.462550,0.189227,2.139052
8,,2021-05-29 15:20:53,6.148 sec,40.0,1.439675,0.187926,2.072663
9,,2021-05-29 15:20:53,6.205 sec,42.0,1.387811,0.185852,1.926019



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Rank,236978.000000,1.000000,0.727712
1,Publisher,30340.964844,0.128033,0.093171
2,Genre,23738.566406,0.100172,0.072896
3,Platform,21019.289062,0.088697,0.064546
4,Year,13571.337891,0.057268,0.041675
